In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

In [ ]:
import os
print(f"Number of CPU cores detected: {os.cpu_count()}")

In [ ]:
# After mounting drive, navigate to your code directory
%cd /content/drive/MyDrive/Raj



/content/drive/MyDrive/Raj


In [ ]:
!pip install albumentations 
!pip install torchmetrics==1.7.3 
!pip install tqdm
!pip install optuna
!pip install tqdm

In [4]:

!cp -r "/content/drive/MyDrive/Raj/data" "/content/"

### Optimisation run for hyperparameters ###

In [ ]:
!python optimsation.py \

In [ ]:
!python train.py \
--data_dir "/content/drive/MyDrive/Raj/data/VOCdevkit/VOC2012" \
--checkpoint_path "/content/drive/MyDrive/Raj/Checkpoints/checkpoint_adamW_final_TEST.pth" \
--epochs 50\
--patience 10


### Traing Unet on seperate Optimizers with the best optimser parameters found from earlier ###

In [ ]:
!python train.py \ 
--data_dir "/content/drive/MyDrive/Raj/data/VOCdevkit/VOC2012" \
--checkpoint_path "/content/drive/MyDrive/Raj/Checkpoints/checkpoint_sgd_final_TEST.pth" \
--epochs 50\
--patience 10


In [ ]:
!python train.py \
--data_dir "/content/drive/MyDrive/Raj/data/VOCdevkit/VOC2012" \
--checkpoint_path "/content/drive/MyDrive/Raj/Checkpoints/checkpoint_sgd_momentum_final_TEST.pth" \
--epochs 50\
--patience 10

In [ ]:
!python train.py \
--data_dir "/content/drive/MyDrive/Raj/data/VOCdevkit/VOC2012" \
--checkpoint_path "/content/drive/MyDrive/Raj/Checkpoints/checkpoint_adam_final_TEST.pth" \
--epochs 50\
--patience 10


In [ ]:
!python train.py \
--data_dir "/content/drive/MyDrive/Raj/data/VOCdevkit/VOC2012" \
--checkpoint_path "/content/drive/MyDrive/Raj/Checkpoints/checkpoint_rsmprop_final_TEST.pth" \
--epochs 50\
--patience 10


### Creating Individual plots for each optimser by running this block repeatedly ###

In [ ]:
# Cell 3: Plotting Script 

# Standard library imports
import os
import json
import numpy as np

# Third-party imports
import torch
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Set Matplotlib backend for non-interactive plotting
matplotlib.use('Agg')

# --- Paths to your Saved Data ---
base_output_dir = "/content/drive/MyDrive/Raj/Checkpoints" 

run_data_dir = os.path.join(base_output_dir, "run_data")

optimizer_name = "SGD_Momentum" # "SGD","SGD_Momentum",etc

history_file = os.path.join(run_data_dir, f"{optimizer_name}_training_history.json")
confusion_matrix_file = os.path.join(run_data_dir, f"{optimizer_name}_confusion_matrix.pt")

# Output directory for the plots themselves
plot_output_dir = os.path.join(base_output_dir, "plots")
os.makedirs(plot_output_dir, exist_ok=True)


# --- Load the Saved Data ---
print(f"Loading training history from: {history_file}")
try:
    with open(history_file, 'r') as f:
        history_data = json.load(f)
    train_losses = history_data["train_losses"]
    val_losses = history_data["val_losses"]
    val_mious = history_data["val_mious"]
    val_pixel_accuracies = history_data["val_pixel_accuracies"]
    print("Training history loaded successfully.")
except FileNotFoundError:
    print(f"Error: Training history file not found at {history_file}.")
    train_losses, val_losses, val_mious, val_pixel_accuracies = [], [], [], []
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {history_file}. File might be corrupted.")
    train_losses, val_losses, val_mious, val_pixel_accuracies = [], [], [], []

print(f"Loading confusion matrix from: {confusion_matrix_file}")
try:
    final_confusion_matrix = torch.load(confusion_matrix_file)
    print("Confusion matrix loaded successfully.")
except FileNotFoundError:
    print(f"Error: Confusion matrix file not found at {confusion_matrix_file}. ")
    final_confusion_matrix = None
except Exception as e:
    print(f"Error loading confusion matrix: {e}")
    final_confusion_matrix = None


# --- Plotting Functions (copied from your utils) ---

def plot_losses(train_losses, val_losses, val_mious, val_pixel_accuracies, save_path, optimizer_name, epoch_offset=0):
  
    #Plots training and validation losses, mIoU, and Pixel Accuracy.
   
    epochs = range(epoch_offset + 1, epoch_offset + len(train_losses) + 1)

    if not epochs:
        print(f"No data to plot for {optimizer_name}.")
        return

    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))
    fig.suptitle(f'Training and Validation Metrics ({optimizer_name})', fontsize=16)

    # Plot Losses
    axes[0].plot(epochs, train_losses, label='Train Loss', marker='o', linestyle='-')
    axes[0].plot(epochs, val_losses, label='Validation Loss', marker='x', linestyle='--')
    axes[0].set_title('Loss over Epochs')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)

    # Plot mIoU and Pixel Accuracy
    axes[1].plot(epochs, val_mious, label='Validation mIoU', marker='o', linestyle='-')
    axes[1].plot(epochs, val_pixel_accuracies, label='Validation Pixel Accuracy', marker='x', linestyle='--')
    axes[1].set_title('Validation Metrics over Epochs')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Metric Value')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.savefig(save_path, dpi=300)
    plt.close(fig)
    print(f"Training metrics plot saved to {save_path}")


def plot_confusion_matrix(cm_tensor, num_classes, class_labels, save_path):

    #Plots a normalized confusion matrix as a heatmap.
    
    if cm_tensor is None:
        print("Confusion matrix tensor is None.")
        return

    cm_numpy = cm_tensor.cpu().numpy()

    cm_normalized = cm_numpy.astype('float') / (cm_numpy.sum(axis=1)[:, np.newaxis] + 1e-6)

    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", ax=ax,
                xticklabels=class_labels, yticklabels=class_labels,
                cbar_kws={'label': 'Normalized Frequency'})

    ax.set_title('Normalized Confusion Matrix')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close(fig)
    print(f"Confusion matrix plot saved to {save_path}")


# --- Execute Plotting ---

pascal_voc_class_labels = [
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
    "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

# Plot training metrics
if train_losses and val_losses and val_mious and val_pixel_accuracies:
    training_plot_path = os.path.join(plot_output_dir, f"{optimizer_name}_training_metrics.png")
    plot_losses(train_losses, val_losses, val_mious, val_pixel_accuracies,
                save_path=training_plot_path,
                optimizer_name=optimizer_name,
                epoch_offset=0)
else:
    print("Not enough data to plot training metrics.")

# Plot confusion matrix
if final_confusion_matrix is not None:
    confusion_matrix_plot_path = os.path.join(plot_output_dir, f"{optimizer_name}_confusion_matrix.png")
    plot_confusion_matrix(final_confusion_matrix,
                          num_classes=len(pascal_voc_class_labels),
                          class_labels=pascal_voc_class_labels,
                          save_path=confusion_matrix_plot_path)
else:
    print("Confusion matrix data not available.")

print("\nPlotting script finished.")

### Creating and storing the combined plots ###

In [ ]:
#====Combined Plot Generation for comparision====
import os
import json
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Set Matplotlib backend for non-interactive plotting
matplotlib.use('Agg')


# --- 1. Configuration for Comparison ---

base_output_dir = "/content/drive/MyDrive/Raj/Checkpoints" 


run_data_dir = os.path.join(base_output_dir, "run_data")


optimizers_to_compare = ["Adam","AdamW", "SGD", "RMSprop","SGD_Momentum"] 

# Output directory for the comparison plots
plot_output_dir = os.path.join(base_output_dir, "comparison_plots")
os.makedirs(plot_output_dir, exist_ok=True)


# --- 2. Data Collection ---
all_optimizer_data = {}

print(f"Loading data for optimizers: {optimizers_to_compare}")
for opt_name in optimizers_to_compare:


    history_file_path = os.path.join(run_data_dir, f"{opt_name}_training_history.json")

    print(f"  Attempting to load {opt_name} history from: {history_file_path}")
    try:
        with open(history_file_path, 'r') as f:
            history_data = json.load(f)
            all_optimizer_data[opt_name] = {
                "train_losses": history_data.get("train_losses", []),
                "val_losses": history_data.get("val_losses", []),
                "val_mious": history_data.get("val_mious", []),
                "val_pixel_accuracies": history_data.get("val_pixel_accuracies", []),
            }
            print(f"  Successfully loaded data for {opt_name}.")
    except FileNotFoundError:
        print(f"  Warning: History file for {opt_name} not found at {history_file_path}")
    except json.JSONDecodeError:
        print(f"  Error: Could not decode JSON for {opt_name} from {history_file_path}. File might be corrupted")
    except KeyError as e:
        print(f"  Error: Missing expected history key {e} in {opt_name}'s history file")

if not all_optimizer_data:
    print("No optimizer data loaded. Cannot generate comparison plots.")
else:
    print("\nData loading complete. Proceeding to plot.")


# --- 3. Plotting Function for Comparison ---

def plot_optimizer_comparison(all_opt_data, plot_save_dir, plot_type):
     #Plots a comparison of a specific metric (loss, mIoU, accuracy) across multiple optimizers.
    if not all_opt_data:
        print(f"No data available for {plot_type} comparison plot.")
        return

    fig, ax = plt.subplots(figsize=(12, 7))

    max_epochs = 0
    for opt_name, data in all_opt_data.items():
        if plot_type == 'loss':
            max_epochs = max(max_epochs, len(data.get('val_losses', [])))
        elif plot_type == 'miou':
            max_epochs = max(max_epochs, len(data.get('val_mious', [])))
        elif plot_type == 'pixel_accuracy':
            max_epochs = max(max_epochs, len(data.get('val_pixel_accuracies', [])))

    if max_epochs == 0:
        print(f"No valid data to plot {plot_type} comparison.")
        plt.close(fig)
        return

    for opt_name, data in all_opt_data.items():
        if plot_type == 'loss':
            metric_data = data.get('val_losses', [])
            title = 'Validation Loss Comparison'
            ylabel = 'Loss'
        elif plot_type == 'miou':
            metric_data = data.get('val_mious', [])
            title = 'Validation mIoU Comparison'
            ylabel = 'mIoU'
        elif plot_type == 'pixel_accuracy':
            metric_data = data.get('val_pixel_accuracies', [])
            title = 'Validation Pixel Accuracy Comparison'
            ylabel = 'Pixel Accuracy'
        else:
            print(f"Invalid plot_type: {plot_type}. Skipping {opt_name}.")
            continue

        epochs = range(1, len(metric_data) + 1)
        if epochs:
            ax.plot(epochs, metric_data, label=f'{opt_name}', marker='o', markersize=4, linestyle='-')


    ax.set_title(title)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid(True)
    ax.set_xlim(1, max_epochs)

    plot_filename = os.path.join(plot_save_dir, f"comparison_val_New_{plot_type}.png")
    plt.tight_layout()
    plt.savefig(plot_filename, dpi=300)
    plt.close(fig)
    print(f"Comparison plot saved for {plot_type}: {plot_filename}")


# --- 4. Execute Comparison Plotting ---
if __name__ == "__main__":
    plot_optimizer_comparison(all_optimizer_data, plot_output_dir, 'loss')
    plot_optimizer_comparison(all_optimizer_data, plot_output_dir, 'miou')
    plot_optimizer_comparison(all_optimizer_data, plot_output_dir, 'pixel_accuracy')

    print("\nAll comparison plots generated.")

Loading sample mask: /content/data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png
Unique class numbers in sample mask '2007_000032.png': [  0  38 147 220]
Minimum class number: 0
Maximum class number: 220


100%|██████████| 1464/1464 [00:03<00:00, 411.54it/s]

Unique labels found in train masks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 255]


100%|██████████| 999/999 [00:21<00:00, 45.70it/s]
